In [7]:
import numpy as np
import os
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import albumentations as A



classes_list = ['sad','friendly','trap','photo','map','bright','deaf','away','help','I','meet','my','name']
num_of_videos = 30
sequence_len = 20
image_height = 240
image_width = 320
number_of_augmentations = 15

In [8]:

# Define the augmentation pipeline
transform = A.ReplayCompose([
    # A.HorizontalFlip(p=0.5),
    # A.RandomBrightnessContrast(p=0.2),
    
    A.OneOf([
        A.ShiftScaleRotate(shift_limit=.2, scale_limit=(-0.3,-0.08), rotate_limit=10, p=0.8,border_mode=cv2.BORDER_CONSTANT),
        A.ShiftScaleRotate(shift_limit=0, scale_limit=(0,0.15), rotate_limit=10, p=0.2,border_mode=cv2.BORDER_CONSTANT)
    ],p=1.0)

 
    # A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=.1),
    # A.OneOf([
    #     # A.CenterCrop(height=(image_height),width=(image_width),p=1.0),
    #     A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.1, rotate_limit=0, p=1.0,border_mode=cv2.BORDER_CONSTANT)
    # ],p=1),
    # A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.4, p=.6),
    # A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH,p=1.0)
    # A.MotionBlur()
    # A.Blur()
    # A.MedianBlur()
])

def load_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # frame = cv2.resize(frame,(image_width,image_height))
        frames.append(frame)
    cap.release()
    return frames

def save_video(frames, output_path):
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()


def augment_video_frames(frames):
    augmented_frames = []
    
    replay = None
    for frame in frames:
        # Apply the augmentations
        if replay is None:
            augmented = transform(image=frame)
            replay = augmented['replay']
        else:
            augmented = A.ReplayCompose.replay(replay, image=frame)
            
        augmented_frames.append(augmented['image'])

    return augmented_frames

def process_video(video_path, output_path):
    # Load video
    video = load_video(video_path)

    # Augment video
    video_aug = augment_video_frames(video)

    # Save video
    save_video(video_aug, output_path)
    
    return np.array(video_aug)



In [9]:
os.makedirs('datasets/augmented',exist_ok=True)

for sign in classes_list:
    counter = 0
    os.makedirs(f'datasets/augmented/{sign}',exist_ok=True)
    for video in os.listdir(f'datasets/original/{sign}'):
        for aug in range(number_of_augmentations):
            process_video(f'datasets/original/{sign}/{video}',f'datasets/augmented/{sign}/{counter}.mp4')
            counter+=1